# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [1]:
import time
start = time.clock()

from distutils.version import StrictVersion
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')


## Env setup

In [2]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

/home/tensorflow/models/research/object_detection/utils/visualization_utils.py:25: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.6/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelapp.py", line 497, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.6/dist-pac

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_FROZEN_GRAPH` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [4]:
# What model to download.
MODEL_NAME = 'duangu-faster-rcnn-20180918'
MODEL_PATH = '/home/tensorflow/models_frozen/' + MODEL_NAME

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_PATH + '/' + 'frozen_inference_graph.pb'


# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('/home/tensorflow/models_my/duangu-faster-rcnn-20180918/config', 'duangu.pbtxt')
print(PATH_TO_LABELS)

NUM_CLASSES = 1

/home/tensorflow/models_my/duangu-faster-rcnn-20180918/config/duangu.pbtxt


## Load a (frozen) Tensorflow model into memory.

In [5]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [6]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Helper code

In [7]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [8]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
## Athena: support multiple directory search
# PATH_TO_TEST_IMAGES_DIR = '/dataspace/electrical_grid_images/bolijveyuanzi/Augmentor-*'
PATH_TO_TEST_IMAGES_DIR = '/dataspace/electrical_grid_images/duangu/Augmentor-skew*'
PATH_TO_SAVE_IMAGES_DIR='/home/tensorflow/result_images'
import glob
jpgfileList = []
for jpgfile in glob.iglob(PATH_TO_TEST_IMAGES_DIR + '/*.jpg'):
    jpgfileList.append(jpgfile)
    print(jpgfile)
TEST_IMAGE_PATHS = jpgfileList
# for jpgfile in os.listdir(PATH_TO_TEST_IMAGES_DIR):
#     if '.jpg' in jpgfile or '.JPG' in jpgfile:
#         jpgfileList.append(jpgfile)
#TEST_IMAGE_PATHS = [PATH_TO_TEST_IMAGES_DIR + '/' + i for i in jpgfileList]

# TEST_IMAGE_PATHS = [os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 104) ]

# Size, in inches, of the output images.
#img = Image.open(TEST_IMAGE_PATHS)
#IMAGE_SIZE = (12, 8)

/dataspace/electrical_grid_images/duangu/Augmentor-skew_top_bottom/original_original_duangu_00034.jpg_aac6b220-2e4a-4fb9-a25d-42aee6dd8ab6.jpg
/dataspace/electrical_grid_images/duangu/Augmentor-skew_top_bottom/original_original_duangu_00033.jpg_365327c5-06b6-44c0-8697-9130603892c4.jpg
/dataspace/electrical_grid_images/duangu/Augmentor-skew_top_bottom/original_original_duangu_00084.jpg_fc76543c-d0c8-42e3-837e-e2c71958617a.jpg
/dataspace/electrical_grid_images/duangu/Augmentor-skew_top_bottom/original_original_duangu_00020.jpg_ec2c30a3-b1f9-4559-b0df-0222e5a287c3.jpg
/dataspace/electrical_grid_images/duangu/Augmentor-skew_top_bottom/original_original_duangu_00032.jpg_7711b657-d0fa-4c83-9021-588100cd8296.jpg
/dataspace/electrical_grid_images/duangu/Augmentor-skew_top_bottom/original_original_duangu_00036.jpg_8782491a-0646-4f23-b9ae-5388dbf8ef80.jpg
/dataspace/electrical_grid_images/duangu/Augmentor-skew_top_bottom/original_original_duangu_00046.jpg_f0b52072-3e52-44ca-a53c-e630ebab4f1a.jpg

In [9]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [ ]:


from PIL import Image

i=0
for image_path in TEST_IMAGE_PATHS:
  i+=1
  ### Athena
  dir=os.path.dirname(image_path)
  outpath=PATH_TO_SAVE_IMAGES_DIR + '/' + MODEL_NAME + '/' + os.path.basename(dir)
  if not os.path.exists(outpath):
    os.makedirs(outpath, 0o755)
    print("outpath:",outpath)

  image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  (im_width, im_height) = image.size
  IMAGE_SIZE = (im_width/100, im_height/100)
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np, detection_graph)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(image_np)
   
  #filename = image_path.split('/')[-1].replace('.JPG', '') + '_faster_rcnn.jpg'
  filename = 'tf_' + os.path.basename(image_path)
  img = Image.fromarray(image_np)
  outfile=outpath + '/' + filename
  img.save(outfile)
  print(i,':',outfile)
#   plt.savefig(PATH_TO_SAVE_IMAGES_DIR + '/' + filename)
  childend = time.clock()
      #print('Every step used time:', childend - childstart)
end = time.clock()
print('Time Used:', end - start)

outpath: /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_top_bottom
1 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_top_bottom/tf_original_original_duangu_00034.jpg_aac6b220-2e4a-4fb9-a25d-42aee6dd8ab6.jpg
2 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_top_bottom/tf_original_original_duangu_00033.jpg_365327c5-06b6-44c0-8697-9130603892c4.jpg
3 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_top_bottom/tf_original_original_duangu_00084.jpg_fc76543c-d0c8-42e3-837e-e2c71958617a.jpg
4 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_top_bottom/tf_original_original_duangu_00020.jpg_ec2c30a3-b1f9-4559-b0df-0222e5a287c3.jpg
5 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_top_bottom/tf_original_original_duangu_00032.jpg_7711b657-d0fa-4c83-9021-588100cd8296.jpg
6 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Aug

/usr/local/lib/python3.6/dist-packages/matplotlib/pyplot.py:522: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


21 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_top_bottom/tf_original_original_duangu_00030.jpg_23e945d6-8e8e-4533-b330-80dd8e617582.jpg
22 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_top_bottom/tf_original_original_duangu_00098.jpg_b2afc966-01df-45eb-a270-c2a2d2eb5fdc.jpg
23 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_top_bottom/tf_original_original_duangu_00080.jpg_a969393c-c349-4268-a9e8-802533bdd492.jpg
24 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_top_bottom/tf_original_original_duangu_00079.jpg_773e2056-b5d4-4847-9881-1bd2030d5461.jpg
25 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_top_bottom/tf_original_original_duangu_00068.jpg_9986b0c0-4872-4bd4-801a-11f0ff5b955f.jpg
26 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_top_bottom/tf_original_original_duangu_00066.jpg_19b4f0c3-8f54-450a-aa4a-ead

70 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_top_bottom/tf_original_original_duangu_00093.jpg_5b8946bc-c61d-44d5-8c3a-3d2c9c3204fd.jpg
71 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_top_bottom/tf_original_original_duangu_00076.jpg_7c6639e0-51a5-4729-ba1e-f706cc193521.jpg
72 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_top_bottom/tf_original_original_duangu_00069.jpg_6e99f1a1-ce29-4ef3-9473-29a8ec6ade70.jpg
73 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_top_bottom/tf_original_original_duangu_00047.jpg_cb6acfba-aa58-4bc8-9efe-6a07633d522a.jpg
74 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_top_bottom/tf_original_original_duangu_00030.jpg_32c940e1-af1c-481c-af3e-5afd7da0d1d9.jpg
75 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_top_bottom/tf_original_original_duangu_00009.jpg_bee8dddc-0705-46de-adc7-af2

119 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_top_bottom/tf_original_original_duangu_00005.jpg_170c6dcb-d37c-434e-b21f-e28e19e2f6c3.jpg
120 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_top_bottom/tf_original_original_duangu_00031.jpg_a92f0e81-03ac-46a3-85c1-0171af901525.jpg
121 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_top_bottom/tf_original_original_duangu_00039.jpg_3f741a93-b927-43fd-a64a-1df5397562bd.jpg
122 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_top_bottom/tf_original_original_duangu_00008.jpg_a30f56d0-51fd-48f7-b189-8e49648afea9.jpg
123 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_top_bottom/tf_original_original_duangu_00069.jpg_08a10d75-ca5d-40ad-bd27-eab5ca6768bb.jpg
124 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_top_bottom/tf_original_original_duangu_00001.jpg_1a6495a2-df59-4833-be

...

413 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_left_right/tf_skew_left_right-duangu_00097_59241f01-97dd-4a62-8506-37e06cadaac5.jpg
414 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_left_right/tf_skew_left_right-duangu_00038_d10068cd-369b-49cd-8907-c7a777306177.jpg
415 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_left_right/tf_skew_left_right-duangu_00007_9ec11e07-c02e-4650-8e2f-20c1fd749182.jpg
416 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_left_right/tf_skew_left_right-duangu_00034_6c9e39f7-2786-41cb-a0d9-eed07974ed27.jpg
417 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_left_right/tf_skew_left_right-duangu_00047_8edae5c6-aebd-4e39-ac27-bfbd27cdffc2.jpg
418 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_left_right/tf_skew_left_right-duangu_00073_ee858f59-d369-4452-b6ff-ce73c841d4bf.jpg
419 : /home/tens

463 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_left_right/tf_skew_left_right-duangu_00072_8e5ff255-6420-4e03-bcf7-5bbf88763e4b.jpg
464 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_left_right/tf_skew_left_right-duangu_00029_195418fd-8920-4b6b-9279-df7eae1f55b3.jpg
465 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_left_right/tf_skew_left_right-duangu_00037_95070ca9-b9e0-493f-b37b-03e6729ef260.jpg
466 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_left_right/tf_skew_left_right-duangu_00079_3c45fc57-00d4-49a2-abd5-63d61de3612e.jpg
467 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_left_right/tf_skew_left_right-duangu_00009_1d31e806-3030-4e92-a0c6-0b6fad5d8c86.jpg
468 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_left_right/tf_skew_left_right-duangu_00098_09b5b508-2092-44fb-a715-c4e01c37a9cf.jpg
469 : /home/tens

513 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_left_right/tf_skew_left_right-duangu_00005_cb896c4f-7515-48be-9e7d-967d6acfcc5c.jpg
514 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_left_right/tf_skew_left_right-duangu_00003_c9e2b484-567a-4970-a6cb-c4e149604de7.jpg
515 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_left_right/tf_skew_left_right-duangu_00040_48af4859-cf14-4bb8-b68f-b6c3a64237c5.jpg
516 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_left_right/tf_skew_left_right-duangu_00055_79f133e2-2fec-4c5b-8f13-1a9b766c394e.jpg
517 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_left_right/tf_skew_left_right-duangu_00006_d0a0f718-cc98-4568-95bb-658e22b01d0c.jpg
518 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_left_right/tf_skew_left_right-duangu_00066_1456c5a0-f9c0-4d5c-b1d9-72bf6fbd1eb5.jpg
519 : /home/tens

563 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_left_right/tf_skew_left_right-duangu_00085_f097dd2b-8119-450e-b00e-5bcf47148307.jpg
564 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_left_right/tf_skew_left_right-duangu_00054_bafd7374-c401-44e0-b7be-f4cb545b0528.jpg
565 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_left_right/tf_skew_left_right-duangu_00016_7d0a21aa-827c-474f-ae21-26b82d97c9b4.jpg
566 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_left_right/tf_skew_left_right-duangu_00098_5cc823c2-8cef-4a48-a383-6a8be455d1d6.jpg
567 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_left_right/tf_skew_left_right-duangu_00072_881271c8-020e-4214-b23b-921f915102a2.jpg
568 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_left_right/tf_skew_left_right-duangu_00089_f00663d7-d1b7-434a-ad95-b20d6f08f74d.jpg
569 : /home/tens

613 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_left_right/tf_skew_left_right-duangu_00061_8b42d7f7-ec10-4d5a-9d98-beb12595e21e.jpg
614 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_left_right/tf_skew_left_right-duangu_00015_313f5481-6134-4e07-ac09-cbcda848c83e.jpg
615 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_left_right/tf_skew_left_right-duangu_00045_005680ad-0fcc-4bef-a740-82443831c5b0.jpg
616 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_left_right/tf_skew_left_right-duangu_00047_e7bfd4e8-f728-4424-905b-e8890df98f9e.jpg
617 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_left_right/tf_skew_left_right-duangu_00017_9e6ac52f-cf3f-4287-b6b0-b58c7c4b14ee.jpg
618 : /home/tensorflow/result_images/duangu-faster-rcnn-20180918/Augmentor-skew_left_right/tf_skew_left_right-duangu_00073_c11ccde0-2657-4ab2-baa2-acbf2ab2dae9.jpg
619 : /home/tens